In [70]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import torch
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
import torchvision.datasets as dset
import torchvision.transforms as T
from sophius.encode import Encoder
import seaborn as sns
from tqdm import tqdm
from sophius.train import train_on_gpu_ex
normalize = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),    
])
cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True, transform=normalize)
cifar_gpu = dload.cifar_to_gpu(cifar10)

encoder = Encoder()

Files already downloaded and verified


In [72]:
from sophius.estimate import LSTMRegressor
from sophius.utils import calc_model_flops

estimator = torch.load('../data/models/estimator_v2.pth').cpu()

def estimate_acc(model_tmpl):
    t = torch.tensor(encoder.model2vec(model_tmpl), dtype=torch.float32)
    return estimator(t).item()

In [73]:
model_gen = ConvModelGenerator(
    in_shape=(3, 32, 32), 
    out_shape=10, 
    conv_num=1, 
    lin_num=1,
)

model_tmpl = model_gen.generate_model_tmpl()
model = model_tmpl.instantiate_model().cuda()

print(model_tmpl)
flops = calc_model_flops(model, model_gen.in_shape)['flops']
print('MFlops', round(flops / 1024 / 1024, 2))

train_params = {
    'val_size': 10000,
    'batch_size': 256,
    'num_epoch': 50,
    'random_seed':42,
    'optimizer': 'AdamW',
    'opt_params': {
        'lr': 1e-3,
    },
    'scheduler': 'ExponentialLR',
    'sch_params': {
        'gamma': 0.95,
    },
}

# res = train_on_gpu_ex(
#     model=model,
#     dataset=cifar_gpu,
#     verbose=True,
#     **train_params,
# )
# print('val_acc* %.3f' % (estimate_acc(model_tmpl)))
# 
# sns.lineplot(res, x='epoch', y='train_acc', label='train')
# sns.lineplot(res, x='epoch', y='val_acc', label='val');

Conv2d       (64, 16, 16)   (5, 5)   (2, 2)  
LeakyReLU    (64, 16, 16)   (0.01)  
AvgPool2d    (64, 15, 15)   (2, 2)   (1, 1)  
GlobalAvgPool2d (64, 1, 1)    
Flatten      64            
Linear       10            

MFlops 2.41


In [74]:
from sophius.db import database, Experiments, Models, ModelEpochs

with database:
    database.create_tables([Experiments, Models, ModelEpochs])

exp_params = {**train_params, **{'in_shape': (3, 32, 32), 'out_shape': 10}}

exp, _ = Experiments.get_or_create(**exp_params)

In [75]:
model_gen = ConvModelGenerator(
    in_shape=(3, 32, 32), out_shape=10, 
    conv_num=3, lin_num=1
)

val_acc_threshold = 0.7

for i in tqdm(range(10)):
    model_tmpl = model_gen.generate_model_tmpl()
    model = model_tmpl.instantiate_model().cuda()
     
    if estimate_acc(model_tmpl) < val_acc_threshold:
        continue
        
    epoch_results = train_on_gpu_ex(
        model=model,
        dataset=cifar_gpu,
        verbose=False,
        **train_params,
    )
    
    model_info = calc_model_flops(model, model_gen.in_shape)
    model = Models.create(
        exp_id = exp.id,
        hash = encoder.model2hash(model_tmpl),
        flops = model_info['flops'],
        macs = model_info['macs'],
        params = model_info['params'],
        val_acc = round(epoch_results.val_acc.iloc[-10:].mean(), 5),
        train_acc = round(epoch_results.train_acc.iloc[-10:].mean(), 5),
        time = round(epoch_results.time.iloc[-1], 1),
    )
    
    
    for _, row in epoch_results.iterrows():
        ModelEpochs.create(
            exp_id = exp.id,
            model_id = model.id,
            epoch = row['epoch'],
            loss = row['loss'],
            train_acc = row['train_acc'],
            val_acc = row['val_acc'],
            time = row['time'],
        )
    

100%|██████████| 10/10 [04:31<00:00, 27.10s/it]


In [76]:
import pandas as pd

with database.connection() as conn:
    df = pd.read_sql('SELECT * from models', conn)
    

In [77]:
df[-10:]

,id,exp_id,hash,flops,macs,params,val_acc,train_acc,time
101,102,1,0082040a08040000002330000082006108080000004260...,632144,310016,55995,0.47172,0.46711,3.3
102,103,1,008082090804000000452000002928000010000001000000,969696,461760,5675,0.34428,0.34559,1.7
103,104,1,00810809080800000022a8000084020908080000001000...,242224,109472,17482,0.37984,0.39256,1.9
104,105,1,0080280610040000400000000022d00001000000,547008,234624,50890,0.50154,0.51152,1.6
105,106,1,0088080a08080000008090060804000000449000400000...,43696,19488,2139,0.30485,0.30672,2.6
106,107,1,008090110808000040000000002950000010000001000000,300000,74688,1642,0.25618,0.24816,2.6
107,108,1,008084090804000000808121200000000028d00001000000,17167488,8568960,248555,0.51439,0.53275,2.6
108,109,1,00810222200000000022c8000080901108080000002550...,7928256,3844032,10442,0.28669,0.28108,4.5
109,110,1,00810111080800000025280001000000,2103552,1032960,97098,0.53743,0.55265,1.3
110,111,2,00804821200000000044a0004000000000808092100200...,75392608,37459200,485387,0.75269,0.99333,270.1


In [78]:
encoder.hash2model('00804821200000000044a000400000000080809210020000400000000024d0000080880a0804000001000000', (3, 32, 32), 10)

Conv2d       (128, 31, 31)  (2, 2)   (1, 1)  
ReLU         (128, 31, 31) 
MaxPool2d    (128, 16, 16)  (3, 3)   (2, 2)  
BatchNorm2d  (128, 16, 16) 
Conv2d       (96, 9, 9)     (6, 6)   (2, 2)  
LeakyReLU    (96, 9, 9)     (0.001) 
BatchNorm2d  (96, 9, 9)    
AvgPool2d    (96, 5, 5)     (3, 3)   (2, 2)  
Conv2d       (96, 2, 2)     (2, 2)   (3, 3)  
PReLU        (96, 2, 2)    
Flatten      384           
Linear       10            